In [5]:
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [123]:
house_df = pd.read_csv("../data/house_combined.csv")
suburb_list = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [124]:
df2 = house_df[['surburb']].drop_duplicates().reset_index().drop('index', axis = 1)

In [125]:
suburb_postcode_df = house_df[['surburb','postcode']].drop_duplicates().reset_index().drop('index', axis = 1)

In [ ]:
list_1 = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        buildings = ox.geometries_from_place(place_name, tags = {'amenity': True, 'landuse':['retail','commercial']})
    except Exception:
        list_1[suburb] = None
        continue
    if len(buildings) == 0:
        list_1[suburb] = None
        continue
    for amenity, sub_df in buildings.groupby('amenity'):
        l[amenity] = len(sub_df)
    list_1[suburb] = l    

In [157]:
attribute_list = {**list_1, **list_2}
attribute_df = pd.DataFrame(attribute_list).T
attribute_df.to_csv('../data/raw/attribute_df.csv')
attribute_df = attribute_df[['bench','cafe','childcare','fast_food','kindergarten','parking','restaurant','place_of_worship','school','university']]

In [158]:
attribute_df

,bench,cafe,childcare,fast_food,kindergarten,parking,restaurant,place_of_worship,school,university
south melbourne,11.0,41.0,4.0,17.0,3.0,30.0,40.0,6.0,1.0,NaN
port melbourne,45.0,32.0,1.0,20.0,1.0,118.0,16.0,3.0,3.0,NaN
albert park,139.0,144.0,6.0,59.0,7.0,289.0,172.0,24.0,12.0,2.0
middle park,7.0,5.0,NaN,2.0,2.0,NaN,4.0,1.0,1.0,NaN
east melbourne,102.0,19.0,NaN,1.0,1.0,55.0,8.0,10.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
tallangatta,NaN,3.0,NaN,1.0,NaN,4.0,NaN,1.0,3.0,NaN
eskdale,NaN,1.0,NaN,NaN,NaN,1.0,NaN,2.0,1.0,NaN
dartmouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
beechworth,1.0,6.0,NaN,NaN,1.0,31.0,4.0,3.0,4.0,NaN


In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list = {}
for suburb in suburb_list['surburb']:
    if(suburb == 'melbourne'):
        continue
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list[suburb] = l    

In [ ]:
leisure_list_2 = {}
for suburb in suburb_list['surburb'][500:]:
    print(suburb)
    l = {}
    place_name = suburb+ ", Victoria, AU" 
    try:
        leisure = ox.geometries_from_place(place_name, tags={'leisure':True})
    except Exception:
        leisure_list_2[suburb] = None
        continue
    if len(leisure) == 0:
        leisure_list_2[suburb] = None
        continue
    for leisure, sub_df in leisure.groupby('leisure'):
        l[leisure] = len(sub_df)
    leisure_list_2[suburb] = l    

In [376]:
leisure_attribute_list = {**leisure_list, **leisure_list_2}
leisure_attribute_list = pd.DataFrame(leisure_attribute_list).T
leisure_attribute_list.to_csv('../data/raw/leisure_df.csv')
leisure_attribute_list = leisure_attribute_list[['garden','park','playground','pitch']]

In [399]:
attribute_conbined = pd.concat([attribute_df,leisure_attribute_list ], axis = 1)

In [400]:
attribute_conbined = attribute_conbined.reset_index().join(suburb_postcode_df).drop('index',axis = 1)
attribute_conbined.to_csv('../data/attribute_conbined.csv')

Download Australia CPI data


In [62]:
xls = pd.ExcelFile('../data/cpi.xlsx')
df1 = pd.read_excel(xls, 'Data')[13:].drop('Unnamed: 0', axis = 1).rename({'Unnamed: 1':'year','Unnamed: 2':'cpi'},axis = 1).reset_index().drop('index',axis = 1)
cpi_list = list(df1['cpi'])
l = []
m = 0
for i in range(2019,2026):
    l.append([i,cpi_list[m]])
    m = m+1
cpi_df = pd.DataFrame(l, columns=['year','cpi'])
cpi_df.to_csv('../data/cpi.csv')

In [3]:
import pandas as pd
import openrouteservice
import math
house_df_combined = pd.read_csv('../data/house_combined.csv')
import ast
lat_list = []
long_list = []
for i in range(len(house_df_combined)):
    if pd.isnull(house_df_combined.geoLocation[i]):
        lat_list.append(None)
        long_list.append(None)
    else:
        latitude = ast.literal_eval(house_df_combined.geoLocation[i])['latitude']
        longitude = ast.literal_eval(house_df_combined.geoLocation[i])['longitude']
        lat_list.append(latitude)
        long_list.append(longitude)
house_df_combined['latitude'] = lat_list
house_df_combined['longitude'] = long_list

In [4]:

# melbourne CBD coordinate (144.961602,-37.813937)
distance_list_6 = []
for i in range(70000,72000):
    print(i)
    if math.isnan(house_df_combined['longitude'][i]) is True:
        distance_list_6.append(None)
    else:
        coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
        client = openrouteservice.Client(key='5b3ce3597851110001cf624852e932154fff48e28bf173d51e105eca') 
        routes = client.directions(coords, profile='driving-car', optimize_waypoints=True, radiuses = 1000)
        distance = routes['routes'][0]['summary']['distance']
        print(distance)
        distance_list_6.append(distance)

70000


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


53001.8
70001
54402.8
70002
47304.5
70003
45141.5
70004
47727.4
70005
49924.0
70006
61635.9
70007
51104.5
70008
45046.4
70009
57863.6
70010
53319.2
70011
56702.6
70012
54862.1
70013
42198.5
70014
61356.4
70015
51378.3
70016
47058.0
70017
49284.7
70018
47521.6
70019
51022.6
70020
58379.1
70021
41101.7
70022
50491.4
70023
44004.0
70024
46249.0
70025
48433.4
70026
34826.4
70027
45192.7
70028
52076.9
70029
45049.5
70030
45185.1
70031
43329.4
70032
30466.9
70033
34507.7
70034
49785.2
70035
57462.0
70036
39000.1
70037
44561.9
70038
42302.3
70039
48473.9
70040


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyon

46062.1
70041
45732.8
70042
43904.5
70043
55575.4
70044
54357.2
70045
33978.5
70046
56605.1
70047
50183.3
70048
37189.2
70049
48595.5
70050
48663.9
70051
55364.1
70052
12966.4
70053
41939.8
70054
17821.2
70055
30596.7
70056
48796.5
70057
34693.0
70058
17821.2
70059
30596.7
70060
35959.2
70061
48821.3
70062
53704.1
70063
52691.3
70064
5688.5
70065
33951.6
70066
50053.4
70067
55612.7
70068
38781.6
70069
52913.6
70070
16891.5
70071
33997.8
70072
34102.3
70073
47997.2
70074
50165.5
70075
50547.9
70076
60952.7
70077
64401.7
70078
49546.2
70079
61022.1
70080
41425.1
70081
53658.0
70082
39543.7
70083
54553.8
70084
55121.7
70085
51980.6
70086
62544.0
70087
45109.5
70088
46798.5
70089
42779.8
70090
49043.9
70091
59728.5
70092
53215.1
70093
36204.7
70094
49912.2
70095
55631.4
70096
41740.1
70097
44756.9
70098
46128.5
70099
50711.8
70100
54736.9
70101
39728.7
70102
99100.4
70103
43879.9
70104
52770.9
70105
48617.4
70106
47799.1
70107
47845.5
70108


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 7th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


46128.5
70109
56522.8
70110
51900.8
70111
55626.7
70112
39106.5
70113
22422.1
70114
55372.0
70115
58442.3
70116
37172.0
70117
37188.8
70118
27742.9
70119
46381.8
70120
51624.9
70121
39000.1
70122
53807.8
70123
44651.3
70124
53160.6
70125
48890.1
70126
48719.5
70127
41585.4
70128
57457.1
70129
31254.9
70130
54047.0
70131
53507.8
70132
47765.0
70133
45494.4
70134
33131.0
70135
53153.5
70136
43677.3
70137
50080.5
70138
16830.2
70139
35490.1
70140
50163.6
70141
45874.8
70142
53408.9
70143
51156.9
70144
52782.4
70145
47749.3
70146
45425.0
70147
49054.2
70148
57919.9
70149
52338.9
70150
52177.3
70151
50549.4
70152
53502.5
70153
80992.9
70154
57895.4
70155
56962.7
70156
83759.8
70157
66257.2
70158
48504.8
70159
93907.6
70160
48405.7
70161
53230.1
70162
63980.9
70163
58865.9
70164
52553.5
70165
163334.6
70166
166558.2
70167
167470.6
70168
166088.7
70169
163751.3
70170
164542.4
70171
163989.3
70172
164605.2
70173
164730.3
70174
178954.5
70175
162219.8
70176
163778.3
70177
162992.6
70178
164628.

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 8th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


52538.1
70262
54847.1
70263
56705.8
70264
58247.7
70265
56634.4
70266
50869.4
70267
55580.2
70268
56402.5
70269
48354.6
70270
51872.8
70271
51422.0
70272
44992.5
70273
46947.0
70274
52010.6
70275
54159.4
70276
51964.6
70277
49912.1
70278
51074.9
70279
56456.1
70280
56657.1
70281
44828.5
70282
56574.8
70283
56258.5
70284
47858.0
70285
56534.8
70286
58070.2
70287
58011.8
70288
51227.0
70289
56855.9
70290
56824.5
70291
54528.8
70292
57667.6
70293
55084.5
70294
51268.2
70295
50532.6
70296
34168.6
70297
49491.2
70298
48187.3
70299
48592.3
70300
54326.1
70301
49567.9
70302
51930.8
70303
52923.6
70304
43412.2
70305
50026.1
70306
48691.1
70307
56433.9
70308
51910.4
70309
28484.8
70310
52556.6
70311
42594.7
70312
51349.9
70313
41118.7
70314
50780.5
70315
51067.7
70316
57137.7
70317
56477.5
70318
53109.5
70319
51041.0
70320
56426.8
70321
50846.5
70322
49716.2
70323
52999.2
70324
54157.7
70325
42447.6
70326
51279.6
70327
51998.8
70328
52284.8
70329
61279.7
70330
55214.5
70331
56465.9
70332
44988.

In [5]:
len(distance_list_6)

2000